In [1]:
from smpl_io import io
import pandas as pd
import numpy as np
import os
import sys

sys.path.append(os.path.dirname("/home/apn/git/crosscheck/crosscheck"))

from crosscheck.data import rel_diff,ratio
from crosscheck.recola2.recola2 import compute as rc_compute
from crosscheck.madgraph.madgraph import compute as mg_compute

In [2]:
def rename(test_df):
    test_df = test_df.rename(columns={1: 'in_1', 2: 'in_2',3:'out_1',4:'out_2', 5:'out_3'})
    test_df = test_df.rename(columns={6:  'p_1_0',  7: 'p_1_1', 8:'p_1_2', 9:'p_1_3'})
    test_df = test_df.rename(columns={10: 'p_2_0', 11: 'p_2_1',12:'p_2_2',13:'p_2_3'})
    test_df = test_df.rename(columns={14: 'p_3_0', 15: 'p_3_1',16:'p_3_2',17:'p_3_3'})
    test_df = test_df.rename(columns={18: 'p_4_0', 19: 'p_4_1',20:'p_4_2',21:'p_4_3'})
    test_df = test_df.rename(columns={22: 'p_5_0', 23: 'p_5_1',24:'p_5_2',25:'p_5_3'})
    test_df = test_df.rename(columns={26: 'alphas'})
    test_df = test_df.rename(columns={27: 'alpha'})
    test_df = test_df.rename(columns={28: 'mu_r'})
    test_df = test_df.rename(columns={29: 'Born'})
    test_df = test_df.rename(columns={30: 'Virt'})
    return test_df

In [3]:
def mask(ol_df):
    #return (ol_df[1] == "-2" )&(ol_df[2] == "2" )&(ol_df[3] == "22" )&(ol_df[4] == "0" )&(ol_df[5] == "0" )
    return (ol_df[3] == "22" )


In [4]:
ol_df = pd.read_csv(io.between("START COMP DEBUG","STOP COMP DEBUG",
                       "/home/apn/git/powheg-box-v2/directphotonjj/testrun-lhc/bench-openloops/working/run-st1-xg1-1.log"
                      ),
            header=None,
            delimiter='\s+',
           )

ol_df = ol_df[(ol_df[0] != "Thanks") & (ol_df[0] != "Eur.Phys.J.")]
ol_df= ol_df.set_index(0)
ol_df = ol_df [ mask(ol_df)]
ol_df = rename(ol_df)
ol_df

In [5]:
rc_df = ol_df.copy()
rc_compute(rc_df,order="NLO")
rc_df['Born'] = rc_df['Born']/rc_df['alphas']**2/rc_df['alpha']*ol_df['alphas']**2*ol_df['alpha']
rc_df['Virt'] = rc_df['Virt']/rc_df['alphas']**3/rc_df['alpha']*ol_df['alphas']**3*ol_df['alpha']/(ol_df['alphas']/2/np.pi)
rc_df

In [ ]:
mg_df = ol_df.copy()
mg_compute(mg_df)
mg_df['Born'] = mg_df['Born']/mg_df['alphas']**2/mg_df['alpha']*ol_df['alphas']**2*ol_df['alpha']
mg_df['Virt'] = mg_df['Virt']/mg_df['alphas']**3/mg_df['alpha']*ol_df['alphas']**3*ol_df['alpha']/(ol_df['alphas']/2/np.pi)
mg_df

In [ ]:
fc_df= pd.read_csv(io.between("START COMP DEBUG","STOP COMP DEBUG",
                       "/home/apn/git/powheg-box-v2/directphotonjj/testrun-lhc/bench-formcalc/working/run-st1-xg1-1.log"),
            header=None,
            delimiter='\s+',
           )
fc_df = fc_df[(fc_df[0] != "Thanks") & (fc_df[0] != "Eur.Phys.J.")]  
fc_df= fc_df.set_index(0)  
fc_df = fc_df [mask(fc_df)]
fc_df = rename(fc_df)
fc_df

In [ ]:
def masked(ol_df):
    #return (ol_df[1] == "-2" )&(ol_df[2] == "2" )&(ol_df[3] == "22" )&(ol_df[4] == "0" )&(ol_df[5] == "0" )
    #return ol_df[(ol_df['in_1'] == "-2" ) & (ol_df['in_2'] == "2")&(ol_df['out_1'] == "22" )&(ol_df['out_2'] == "4" )&(ol_df['out_3'] == "-4" )]
    return ol_df
m_fc_df = masked(fc_df)
m_ol_df = masked(ol_df)
m_mg_df = masked(mg_df)
m_rc_df = masked(rc_df)


In [ ]:
print(rel_diff(m_fc_df, m_ol_df,m_mg_df, m_rc_df,column="Virt",labels=["fc", "ol","mg","rc"]))
print(rel_diff(m_ol_df,m_mg_df, m_rc_df,column="Virt",labels=[ "ol","mg","rc"]))
print(rel_diff(m_mg_df, m_rc_df,column="Virt",labels=[ "mg","rc"]))

In [ ]:
print(ratio(m_fc_df, m_ol_df,m_mg_df, m_rc_df,column="Virt",labels=["fc", "ol","mg","rc"],logy=False))
print(ratio(m_ol_df,m_mg_df, m_rc_df,column="Virt",labels=[ "ol","mg","rc"],logy=False))
print(ratio(m_mg_df, m_rc_df,column="Virt",labels=[ "mg","rc"],logy=False))